# Monte-Carlo-Simulation

© Thomas Robert Holy 2019
<br>
Version 0.2.1

## Grundlegende Einstellungen:

Zunächst müssen die notwendigen Pakete (auch Module) importiert werden, damit auf diese zugegriffen werden kann. 

In [ ]:
import pandas as pd # Programmbibliothek die Hilfsmittel für die Verwaltung von Daten und deren Analyse anbietet
import scipy.stats as st # SciPy ist ein Python-basiertes Ökosystem für Open-Source-Software für Mathematik, Naturwissenschaften und Ingenieurwissenschaften
from scipy.stats import rankdata, norm  
from scipy import array, linalg, dot
import random # Dieses modul wird verwendet um Zufallszahlen zu ziehen
import numpy as np # Programmbibliothek die eine einfache Handhabung von Vektoren, Matrizen oder generell großen mehrdimensionalen Arrays ermöglicht
import math # Dieses Modul wird verwendet um Skalardaten zu berechnen, z. B. trigonometrische Berechnungen.
import operator # Programmbibliothek, welche die Ausgaben übersichtlicher gestaltet
import matplotlib.pyplot as plt # Programmbibliothek die es erlaubt mathematische Darstellungen aller Art anzufertigen
import matplotlib.patches as mpatches

Anschließend werden Einstellungen definiert, die die Formatierung der Ausgaben betreffen.
Hierfür wird das Modul `operator` genutzt.
Außerdem wird die Größe der Grafiken modifiziert, welche später angezeigt werden sollen.

In [ ]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

SCREEN_WIDTH = 114
centered = operator.methodcaller('center', SCREEN_WIDTH)

plt.rcParams["figure.figsize"] = 15,15

## Monte-Carlo Simulation:

### Variablen spezifizieren

Im ersten Schritt werden die für die Simulation notwenigen Variablen definiert.

In [ ]:
##########################################################################
#-------------------------------------------------------------------------
# Anzahl Simulationsdurchläufe
n = 10000

#----------------------------------------------------
# Neue Randverteilungen (Gleichverteilung)
rand_x = [10,20]
rand_y = [8,22]

#----------------------------------------------------
# Varianzen und Korrelation(en)
var_x = 4 
var_y = 9

corr_list = [0]
std_list = [math.sqrt(var_x),math.sqrt(var_y)]

#----------------------------------------------------
# Erwartungswerte
mu = [2, 3]
#-------------------------------------------------------------------------
##########################################################################

### Funktionen definieren

Als nächstes werden Funktionen definiert. Die erste Funktion berechnet aus den gegeben Varianzen und dazugehörigen Korrelationen die Varianz-Kovarianz-Matrix, die Zweite legt ein Array mit den gegebenen Varianzen an und die Dritte führt die Cholesky-Zerlegung auf der Grundlage Varianz-Kovarianz-Matrix durch.

In [ ]:
#---------------------------------------------------------------------------------------------------------------
# Varianz-Kovarianz-Matrix erstellen

def var_covar_matrix_func(std_list,corr_list):
    global var_covar
    
    i = 0
    j = 0
    counter_0 = 0
    counter_1 = 0    
    len_std_list = len(std_list)
    array = [[0] * len_std_list] * len_std_list    
    val_list = []
    
    for i in range(0,len_std_list):
        for j in range(0,len_std_list):
            if i == j:
                val = std_list[i] * std_list[i]
                val_list.append(val)
            if i < j:
                val = (std_list[i] * std_list[j] * corr_list[counter_0])
                counter_0 += 1
                val_list.append(val)
            if i > j:
                val = (std_list[i] * std_list[j] * corr_list[counter_1])
                counter_1 += 1
                val_list.append(val)
    var_covar = np.array(val_list).reshape(len_std_list,len_std_list)   
    
#---------------------------------------------------------------------------------------------------------------
# Varianz-Array erstellen

def var_func():
    global var_list
    
    var_list = []    
    for i in range(0,len(std_list)):
        var = np.power(std_list[i],2)
        var_list.append(var)

#---------------------------------------------------------------------------------------------------------------
# Cholesky-Zerlegung durchführen

def cholesky_func():
    global cholesky
    
    cholesky = linalg.cholesky(var_covar, lower=True)

#---------------------------------------------------------------------------------------------------------------   
# Funktionen aufrufen

var_func()
var_covar_matrix_func(std_list,corr_list)
cholesky_func()

### Funktion für die Monte-Carlo-Simulation definieren

Nun wird eine Funktion definiert, in welcher die Simulation durchgeführt wird. 
Dabei werden zunächst zwei gleichverteilte Pseudo-Zufallsvariablen generiert, welche anschließend in unabhängige standardnormalverteilte Zufallsvariablen transformiert werden.
Letztere werden unter Anwendung der Cholesky-Zerlegung in abhängige standardnormalverteilte Zufallsvariablen überführt.
Danach werden diese unter Anwendung der oben gegebenen Standardabweichungen und der gegebenen Erwartungswerte in Realisationen der Gauss-Copula überführt, bevor auf diese anschließend die neuen Randverteilungen angewendet werden.
Zum Schluss wird die Summe aus beiden Realisationen gebildet, welche den Portfolio-Wert ergibt.

In [ ]:
total_standard_norm_ab_list = []
total_realisation_cop_list = []
total_xy_list = []
total_summe_liste = []

def copula_sim(n, rand_x, rand_y, mu, full_log):
    global total_standard_norm_ab_list
    global total_realisation_cop_list
    global total_xy_list
    global total_summe_liste
    
    for i in range(0,n):        
        #---------------------------------------------------------------------------------------------------------------
        # Gleichverteilte Zufallsvariablen ziehen
        
        n = 2
        random_ZV_list = [] 
        
        for i in range(0,n):
            x = random.random()
            random_ZV_list.append(x)
        
        #---------------------------------------------------------------------------------------------------------------
        # Transformation der gleichverteilten Zufallsvariablen in unabhängige standardnormalverteilte Zufallsvariablen

        standard_norm_list = norm.ppf(random_ZV_list)        
        
        #---------------------------------------------------------------------------------------------------------------
        # Transformation in standardnormalvertzeilte abhängie ZV
        
        standard_norm_ab_list = []
        counter_cholesky_0 = 0
        counter_cholesky_1 = 0
        counter_mu = 0
        counter_standard_norm_list = 0

        for i in range(0,2):
            a = cholesky[counter_cholesky_0][counter_cholesky_1] * standard_norm_list[counter_standard_norm_list]
            counter_cholesky_1 += 1
            counter_standard_norm_list += 1

            b = cholesky[counter_cholesky_0][counter_cholesky_1] * standard_norm_list[counter_standard_norm_list]
            counter_cholesky_0 += 1  
            counter_cholesky_1 = 0
            counter_standard_norm_list = 0  

            r = a  + b + mu[counter_mu]
            standard_norm_ab_list.append(r)
            counter_mu += 1

        #---------------------------------------------------------------------------------------------------------------
        # Transformation Realisationen der Gauss-Copula

        counter = 0
        realisation_cop_list = []    
        
        for i in range(0,2):
            r_cop = (standard_norm_ab_list[counter] - mu[counter]) / math.sqrt(var_list[counter])   
            p_value = st.norm.cdf(r_cop)
            realisation_cop_list.append(p_value)
            counter += 1
            
        #---------------------------------------------------------------------------------------------------------------
        # Gemeinsame Verteilung: Übertragung der neuen Ränder auf die ermittelte Abhängigkeitsstruktur          

        x = (rand_x[0]-rand_x[1]) * realisation_cop_list[0] + rand_x[1]
        y = (rand_y[0]-rand_y[1]) * realisation_cop_list[1] + rand_y[1]
        summe = x + y        
        xy = (x,y)
        
        if full_log == True:
            total_standard_norm_ab_list.append(standard_norm_ab_list)
            total_realisation_cop_list.append(realisation_cop_list)
            total_xy_list.append(xy)
        total_summe_liste.append(summe)
        
copula_sim(n, rand_x, rand_y, mu, full_log=True)

# Graphische Ausgaben:

## Realisationen abhängiger bivariat normalverteilter Zufallsvariablen

In [ ]:
counter_1 = 0
counter_2 = 0
x_liste = []
y_liste = []

for i in range(0,len(total_standard_norm_ab_list)):
    x = total_standard_norm_ab_list[counter_1][0]
    y = total_standard_norm_ab_list[counter_1][1]
    x_liste.append(x)
    y_liste.append(y)
    counter_1 += 1
    counter_2 += 1

plt.title('Realisationen abhängiger bivariat normalverteilter Zufallsvariablen')
plt.scatter(x_liste, y_liste)
def plot_func():
    plt.grid()
    plt.xlabel('Realisation X')
    plt.ylabel('Realisation Y')
    plt.axhline(0, color='black')
    plt.axvline(0, color='black')
plot_func()
plt.show()

## Realisationen Gauss-Copula

In [ ]:
counter_1 = 0
counter_2 = 0
x_liste = []
y_liste = []

for i in range(0,len(total_realisation_cop_list)):
    x = total_realisation_cop_list[counter_1][0]
    y = total_realisation_cop_list[counter_1][1]
    x_liste.append(x)
    y_liste.append(y)
    counter_1 += 1
    counter_2 += 1

plt.title('Realisationen Gauss-Copula')
plt.scatter(x_liste, y_liste)
plot_func()
plt.show()


## Realisationen gleichverteilte X, Y mit Gauss-Copula

In [ ]:
counter_1 = 0
counter_2 = 0
x_liste = []
y_liste = []

for i in range(0,len(total_xy_list)):
    x = total_xy_list[counter_1][0]
    y = total_xy_list[counter_1][1]
    x_liste.append(x)
    y_liste.append(y)
    counter_1 += 1
    counter_2 += 1    
    
plt.title('Realisationen gleichverteilte X, Y mit Gauss-Copula')
plt.scatter(x_liste, y_liste)
def xy_lim_func():
    left, right = plt.xlim()
    plt.xlim((left, right))   
    plt.xlim(left, right)     
    left, right = plt.ylim()
    plt.ylim((left, right)) 
    plt.ylim(left, right)     
xy_lim_func()   
plot_func()   
plt.show()

## Histogramm X+Y, Realisationen gleichverteilte X, Y mit Gauss-Copula

In [ ]:
bins = 71
data = total_summe_liste
plt.hist(data, bins=bins)
plt.grid()
plt.title('Histogram X+Y, Realisationen gleichverteilte X,Y mit Gauss-Copula')
plt.xlabel('PF-Realisation')
plt.ylabel('Anzahl')
plt.show()

## Verteilung X+Y, Realisationen gleichverteilte X, Y mit Gauss-Copula

In [ ]:
H, X1 = np.histogram(total_summe_liste, bins=n, density=True)
def hist_func():
    dx = X1[1] - X1[0]
    F1 = np.cumsum(H)*dx
    plt.plot(X1[1:], F1)
hist_func()

def verteilung_func():
    plt.title('Verteilung X+Y, Realisationen gleichverteilte X,Y mit Gauss-Copula')
    blue_patch = mpatches.Patch(color='blue', label='Monte-Carlo-Simulation(en)')
    plt.legend(handles=[blue_patch], loc='upper left')
    plt.grid()
    plt.xlabel('PF-Realisation')
    plt.ylabel('Wahrscheinlichkeit')
    left, right = plt.xlim()
    plt.xlim((left, right))
    plt.xlim(left, right)
    plt.axhline(0, color='black')
    plt.axvline(0, color='black')
    plt.show()
verteilung_func()

## Risikomaße schätzen:

### Value at Risk

Um den Value at Risk für die Monte-Carlo-Simulation zu bestimmen werden die Portfolio-Realisationen zunächst der Größe nach sortiert, wobei dieser Schritt gleichzeitig der Ausgangspunkt für die Berechnung des Conditional Value at Risk und des Power-Sepktralen Risikomaßes ist.
Anschließend wird das Alpha-Quantil der Verlustfunktion bestimmt, indem der Parameter "alpha" mit der Länge der Liste "RM_list" multipliziert wird. Der so ermittelte Wert gibt die Position in der Liste "RM_list" an, an welcher sich der Value at Risk zum Konfidenzniveau "alpha" befindet.

In [ ]:
RM_list = sorted(total_summe_liste)

def VaR_func(alpha):
    global VaR
    item = int((alpha * len(RM_list))) - 1
    VaR = -(RM_list[item])

### Conditional Value at Risk

Der Conditional Value at Risk wird grundsätzlich wie der Value at Risk bestimmt, wobei hier jedoch der Mittelwert über alle Realisationen bis zum Alpha-Quantil gebildet wird. 
Daher wird hier nach der Positionsbestimmung die Liste "CVaR_list" mit denjenigen Realisationen aus der Liste "RM_list" gefüllt, welche den Bereich von der kleinsten Realisation bis zum Alpha-Quantil abdecken. 
Die Summe dieser Liste wird anschließend durch die Anzahl ihrer Elemente geteilt.

In [ ]:
def CVaR_func(alpha):
    global CVaR
    item = int((alpha * len(RM_list)))
    CVaR_list = RM_list[0:item]     
    CVaR = -(np.sum(CVaR_list) / len(CVaR_list))

### Power-Spektrales Risikomaß

Für das Power-Spektrale Risikomaß ergibt sich der Erwartungswert aus dem Mittelwert der "RM_list" (der Mittelwert der Portfolio-Realisationen) und das Risiko ergibt sich aus dem Matrixprodukt der transponierten "RM_list" mit der "subj_ws_list", welche subjektive Wahrscheinlichkeiten beinhaltet.
Die Elemente letzterer Liste werden berechnet, indem die Laufvariable jeder Realisation in der geordneten Statisitk "RM_list" (hier: 1 bis 10.000) durch die Gesamtanzahl der Realisationen (hier: 10.000) geteilt und dann mit "gamma" potenziert wird (daher heißt es Power-Sepktrales Risikomaß).
Dabei ist bei jeder Berechnung der jeweils vorher errechnete Wert zu subtrahieren.

In [ ]:
def power_func(gamma):
    global EW
    global risk
    EW = np.mean(RM_list)

    subj_ws_list = []
    counter1 = len(RM_list)
    counter2 = len(RM_list) - 1
    for i in RM_list:
        subj_ws = (np.power((counter1 / len(RM_list)), gamma)) - (np.power((counter2 / len(RM_list)), gamma)) 
        counter1 -= 1
        counter2 -= 1
        subj_ws_list.append(subj_ws)
    subj_ws_list = subj_ws_list[::-1]
    risk = np.matmul(np.transpose(RM_list), subj_ws_list)

### Parameterfestlegung und Aufruf der Funktionen

In [ ]:
##########################################################################
#-------------------------------------------------------------------------
alpha = 0.05

VaR_func(alpha)
print('#------------------------------------------------------------------------------------------------------------------#')
print('|' + centered('Der VaR beträgt: ' + str(VaR) + '.') + '| ')
#-------------------------------------------

alpha = 0.05

CVaR_func(alpha)
print('#------------------------------------------------------------------------------------------------------------------#')
print('|' + centered('Der CVaR beträgt: ' + str(CVaR) + '.') + '| ')
#-------------------------------------------

gamma = 0.5

power_func(gamma)
print('#------------------------------------------------------------------------------------------------------------------#')
print('|' + centered('Power-Spektrales Risikomaß bei der Monte-Carlo-Simulation:') + '| ')
print('#------------------------------------------------------------------------------------------------------------------#')
print('|' + centered('Der Erwartungswert beträgt: ' + str(EW) + '.') + '| ')
print('|' + centered('Das Risiko beträgt: ' + str(risk) + '.') + '| ')
print('#------------------------------------------------------------------------------------------------------------------#')

#-------------------------------------------------------------------------
##########################################################################

## Inkonsistenz Monte-Carlo-Simulation

Ein Kritikpunkt an der Monte-Carlo-Simulation ist, dass das Ergebnis des Verfahrens großen Schwankungen unterliegen kann, sofern nur wenige Realisationen in einem Simulationslauf simuliert werden.
Dies soll die folgende Grafik veranschaulichen.
Dabei kann sowohl die Anzahl der Simulationsläufe als auch die Anzahl der in jeder Simulation durchgeführten Simulationen variiert werden.

In [ ]:
total_summe_liste = []

#-------------------------------------------
# Variablen für den Simulationslauf

rand_x = [10,20]
rand_y = [8,22]
var_x = 4 
var_y = 9
corr_list = [0]
std_list = [math.sqrt(var_x),math.sqrt(var_y)]
mu = [2, 3]

#-------------------------------------------
# Funktionsaufruf

var_func()
var_covar_matrix_func(std_list,corr_list)
cholesky_func()

#-------------------------------------------
# Paramaeter Risikomaße

alpha = 0.05
gamma = 0.5

##########################################################################
#-------------------------------------------------------------------------
# Wiederholungen der Simulationen

runs_sim = 100 # Legt die Anzahl der Durchläufe einer Simulation fest
runs_func = 10 # Legt fest, wie viele Simulationen durchgeführt werden

#-------------------------------------------------------------------------
##########################################################################

RM_VaR_list = []
RM_CVaR_list = []
RM_PSRM_list = []
mega_summe_list = []

for i in range(0,runs_func):
    copula_sim(runs_sim, rand_x, rand_y, mu, full_log=False)
    mega_summe_list += total_summe_liste
    RM_list = sorted(total_summe_liste)    
    VaR_func(alpha)
    RM_VaR_list.append(VaR)    
    CVaR_func(alpha)
    RM_CVaR_list.append(CVaR)
    power_func(gamma)
    RM_PSRM_list.append(risk)
    RM_list = []
    total_summe_liste = []
    
RM_frame = pd.DataFrame()
RM_frame['VaR'] = RM_VaR_list
RM_frame['VaR-Change'] = RM_frame['VaR'].pct_change()
RM_frame['CVaR'] = RM_CVaR_list
RM_frame['CVaR-Change'] = RM_frame['CVaR'].pct_change()
RM_frame['Power'] = RM_PSRM_list
RM_frame['Power-Change'] = RM_frame['Power'].pct_change()

Min_Max_VaR = (min(RM_VaR_list), max(RM_VaR_list))
Min_Max_CVaR = (min(RM_CVaR_list), max(RM_CVaR_list))
Min_Max_PSRM = (min(RM_PSRM_list), max(RM_PSRM_list))

print('#------------------------------------------------------------------------------------------------------------------#')
print('|' + centered('[INFO] Der Dataframe mit den auf den ' + str(runs_sim) + ' beruhenden Risikomaßen ergibt sich wie folgt: ') + '| ')
print('#------------------------------------------------------------------------------------------------------------------#')
print(RM_frame)
print('#------------------------------------------------------------------------------------------------------------------#')

print('|' + centered('Nach ' + str(runs_func) + ' Simulationsläufen mit je ' + str(runs_sim) + ' Durchläufen beträgt der kleinste VaR ' + str(round(Min_Max_VaR[0],2)) +', der größte ' + str(round(Min_Max_VaR[1],2)) + ' (\u0394 = ' + str((round((float(Min_Max_VaR[0]/Min_Max_VaR[1])-1)*100,2))) + '%).') + '| ')
print('|' + centered('Nach ' + str(runs_func) + ' Simulationsläufen mit je ' + str(runs_sim) + ' Durchläufen beträgt der kleinste CVaR ' + str(round(Min_Max_CVaR[0],2)) +', der größte ' + str(round(Min_Max_CVaR[1],2)) + ' (\u0394 = ' + str((round((float(Min_Max_CVaR[0]/Min_Max_CVaR[1])-1)*100,2))) + '%).') + '| ')
print('|' + centered('Nach ' + str(runs_func) + ' Simulationsläufen mit je ' + str(runs_sim) + ' Durchläufen beträgt das kleinste P-SRM ' + str(round(Min_Max_PSRM[0],2)) +', das größte ' + str(round(Min_Max_PSRM[1],2)) + ' (\u0394 = ' + str((round((float(Min_Max_PSRM[0]/Min_Max_PSRM[1])-1)*100,2))) + '%).') + '| ')
print('#------------------------------------------------------------------------------------------------------------------#')

#-------------------------------------------

counter_0 = 0
counter_1 = runs_sim
array = []
for i in range(0,runs_func):
    x = mega_summe_list[counter_0:counter_1]
    array.append(x)
    counter_0 += runs_sim
    counter_1 += runs_sim

#-------------------------------------------

for items in array:
    values_PF = items
    bins = runs_sim
    
    H, X1 = np.histogram(values_PF, bins, density=True)
    hist_func()

verteilung_func()